# Wyznaczenie pozycji względnej na podstawie danych obserwacyjnych GPS

### Instalacja georinex

In [2]:
!pip install georinex

### Funkcja XYZGPS:

In [1]:
def XYZGPS(v, ts):
    # stałe
    u = 3.986005*10**14 # iloczyn stałej grawitacji i masy Ziemi
    omega_e = 7.2921151467*10**-5 

    tk = ts - v.Toe # epoka końcowa dla parametrów efemerydalnych

    a = v.sqrtA**2 # duża półoś orbity
    n0 = sqrt(u/a**3) # ruch średni
    n = n0+v.DeltaN # skorygowany ruch średni
    Mk = v.M0+n*tk # średnia anomalia

    E0 = Mk
    E1 = 100

    while all(abs(E0-E1)) > 10**-15:
        E1 = E0
        E0 = Mk + v.Eccentricity*sin(E0)

    L = sqrt(1-v.Eccentricity**2)*sin(E0)
    M = cos(E0)-v.Eccentricity
    fi = arctan2(L,M)

    vk = fi # prawdziwa anomalia

    fik = vk+v.omega # argument szerokości
    
    #perturbacje harmoniczne
    duk = v.Cus*sin(2*fik)+v.Cuc*cos(2*fik) # argument korekty szerokości
    drk = v.Crs*sin(2*fik)+v.Crc*cos(2*fik) # promień
    dik = v.Cis*sin(2*fik)+v.Cic*cos(2*fik) # nachylenie

    uk = fik + duk # poprawiony argument
    rk = a*(1-v.Eccentricity*cos(E0)) + drk # promień
    ik = v.Io+dik+v.IDOT*tk # nachylenie
    
    # pozycja w płaszczyźnie orbitalnej
    xkp = rk*cos(uk) 
    ykp = rk*sin(uk)

    omk = v.Omega0 + (v.OmegaDot-omega_e) * tk - omega_e * v.Toe # poprawiona długość węzła wstępującego

    # współrzędne odbiornika
    xk = xkp*cos(omk)-ykp*cos(ik)*sin(omk) 
    yk = xkp*sin(omk) + ykp*cos(ik)*cos(omk)
    zk = ykp*sin(ik)

    return xarray.DataArray([xk.values, yk.values, zk.values])

### Funckja wykonująca iteracje

In [2]:
def obl_iter(dt0, x0, y0, z0, x, y, z):
    
    p0 = sqrt((x-x0)**2 + (y-y0)**2 + (z-z0)**2).to_numpy()
    A = np.array([-(x-x0)/p0, -(y-y0)/p0, -(z-z0)/p0, [1 for i in range(5)]]).T
    L = P - p0
    x = lstsq(A, L, rcond = -1)[0]
    xp, yp, zp, dt0 = [x0, y0, z0, dt0] + x
    dx, dy, dz = xp - x0, yp - y0, zp - z0
    x0, y0, z0, = xp, yp, zp
    
    return x0, y0, z0, dt0, dx, dy, dz, A

## SKRYPT GŁÓWNY

In [3]:
import xarray
import georinex as gr
from pandas import Timestamp as tsp
import numpy as np
from numpy import sin, cos, sqrt, arctan2, pi, arctan, abs
from numpy.linalg import lstsq

# Stale
n = 4
c = 299792458

# Wczytanie danych obserwacyjnych i nawigacyjnych
obs = gr.load(r'./KRAW135N.11o')
nav = gr.load(r'./KRAW135N.11n')

# [2] Dane obserwacyjne:
P = obs.isel({'time':n-1}).C1.dropna('sv').isel({'sv': [i for i in range(5)]}) #pomierzone pseudoodległości
PRN = list(P.get_index('sv')) # lista numerów PRN 05, 07, 08, 15, 19

epoka = tsp(P.time.values) # epoka odbioru sygnału
x0,y0,z0 = obs.position # przybliżone współrzędne odbiornika odczytane z nagłówka zbioru KRAW135N.11o

# [3] Obliczenie epoki odbioru sygnału tr, jako liczby sekund od początku tygodnia GPS
GPSD = int(epoka.to_julian_date() - tsp('1980-01-6T0:00:00').to_julian_date())%7 # numer dnia w tygodniu GPS
tr = GPSD*86400 + epoka.hour * 3600 + epoka.minute*60 + epoka.second # Epoka odbioru sygnału

# [4] Obliczanie epok emisji sygnału
ts = tr - (P/c)

# [5] Wczytanie danych efemerydalnych  najbliższych epoce tr  (KRAW135N_SM.11n)
epoka_nav = epoka.round('2H')
v = nav.loc[{'time': epoka_nav, 'sv': PRN}]

# [6] Obliczenie współrzędnych satelitów na epokę emisji sygnału
x, y, z = XYZGPS(v, ts)

# [7] Obliczenie poprawek zegarów satelitów
dt = v.SVclockBias + v.SVclockDrift*(ts - v.Toe) + v.SVclockDriftRate*(ts - v.Toe)**2

# [8] Obliczenie poprawionych pseudoodległości
P += c*dt
P = P.to_numpy()

# [9] obliczanie iteracyjne
dt0 = 0
xp, yp, zp = x0,y0,z0
for i in range(3):
    x0, y0, z0, dt0, dx, dy, dz, A = obl_iter(dt0, x0, y0, z0, x, y, z)
    
print(f'współrzędne x, y, z po 2 iteracji wynoszą [m]: {round(x0,4)}, {round(y0,4)}, {round(z0,4)}')
print(f'Różnica dx, dy, dz między współrzędnymi przybliżonymi i obliczonymi w 2 iteracji w [m]:\n {round(x0-xp, 4)}, {round(y0-yp, 4)}, {round(z0-zp, 4)}')

współrzędne x, y, z po 2 iteracji wynoszą [m]: 3856950.8382, 1397784.684, 4867745.3755
Różnica dx, dy, dz między współrzędnymi przybliżonymi i obliczonymi w 2 iteracji w [m]:
 14.677, 34.2078, 25.9448


## Ćwiczenie o4

In [4]:
from numpy.linalg import inv
A
xp,yp,zp

# Współczynniki DOP w układzie geocentrycznym
Q = inv(A.T@A)
GDOP = sqrt(Q[0,0]+Q[1,1]+Q[2,2]+Q[3,3]) # geometryczny
PDOP = sqrt(Q[0,0]+Q[1,1]+Q[2,2]) # czasowy
TDOP = sqrt(Q[3,3]) # przestrzenny
print(PDOP)

# Wspołczynniki DOP w układzie topocentrycznym 
a = 6378137
b = 6356752.31425
e = sqrt((a**2-b**2)/a**2)
ep = sqrt((a**2-b**2)/b**2)
p = sqrt(xp**2+yp**2)
om = arctan(zp*a/p*b)
fi = arctan((zp+ep**2*b*sin(om)**3)/(p-e**2*a*cos(om)**3))
lamb = arctan(yp/xp)

Q = Q[:3,:3]

R = np.array([[-sin(fi)*cos(lamb), -sin(fi)*(sin(lamb)), cos(fi)],
     [-sin(lamb), cos(lamb), 0], 
    [cos(fi)*cos(lamb), cos(fi)*sin(lamb), sin(fi)]])

Qn = R@Q@R.T

GDOP = sqrt(Qn[0,0]+Qn[1,1]+Qn[2,2])
HDOP = sqrt(Qn[0,0]+Qn[1,1])
VDOP = sqrt(Qn[2,2])

GDOP, HDOP, VDOP

2.806813142796528


(2.806813142796528, 1.2734995229855575, 2.501279469297839)

In [38]:
A

array([[-0.05900322, -0.66569879, -0.74388422,  1.        ],
       [-0.58816965,  0.02018203, -0.80848571,  1.        ],
       [ 0.67454917, -0.29355448, -0.67735455,  1.        ],
       [-0.27109574,  0.64096604, -0.71810141,  1.        ],
       [-0.98862944,  0.14053438,  0.05349696,  1.        ]])